In [250]:
import pandas as pd

In [251]:
FILE='final_properties.csv'
#FILE='svc1.csv'
data=pd.read_csv(FILE,low_memory=False)
data.drop(['url','adress','subtype_of_property','id'],inplace=True,axis=1)
data=data.fillna(0)

data.shape
data.head(10)

,price,type_of_property,bedroomCount,land_surface,kitchen_type,fireplaceExists,hasTerrace,terraceSurface,hasGarden,gardenSurface,surface_land_area,netHabitableSurface,facadeCount,hasSwimmingPool,isnewly_built
0,185000.0,HOUSE,3.0,0.0,0,0,1.0,0.0,0,0.0,0.0,115.0,2.0,0.0,1
1,189000.0,HOUSE,3.0,251.0,USA_INSTALLED,0,1.0,0.0,True,150.0,251.0,158.0,3.0,0.0,1
2,189900.0,HOUSE,3.0,389.0,0,0,1.0,15.0,True,304.0,389.0,243.0,3.0,0.0,1
3,194900.0,HOUSE,3.0,414.0,0,0,1.0,15.0,True,329.0,414.0,243.0,4.0,0.0,1
4,195000.0,HOUSE,3.0,190.0,NOT_INSTALLED,0,1.0,17.0,True,123.0,190.0,180.0,3.0,0.0,1
5,195000.0,HOUSE,4.0,210.0,0,0,1.0,1.0,True,1.0,210.0,0.0,2.0,0.0,1
6,195000.0,HOUSE,3.0,35.0,0,0,1.0,1.0,True,1.0,35.0,220.0,3.0,0.0,1
7,220000.0,HOUSE,2.0,145.0,0,0,1.0,18.0,0,0.0,145.0,118.0,0.0,0.0,1
8,175000.0,APARTMENT,1.0,0.0,INSTALLED,0,1.0,4.0,0,0.0,0.0,50.0,0.0,0.0,1
9,180000.0,APARTMENT,2.0,0.0,USA_HYPER_EQUIPPED,0,1.0,5.0,True,2500.0,0.0,100.0,4.0,0.0,1


In [252]:

data = data.drop(data[data['bedroomCount'] > 7].index)
data = data.drop(data[data['type_of_property'] == 'APARTMENT_GROUP'].index)
data = data.drop(data[data['type_of_property'] == 'HOUSE_GROUP'].index)
data = data.drop(data[data['bedroomCount'] > 7].index)
data = data.drop(data[data['bedroomCount'] == 0].index)
data = data.drop(data[data['price'] <= 100000].index)

data = data.drop(data[data['netHabitableSurface'] == 0].index)
data = data.drop(data[data['netHabitableSurface'] > 600].index)



data["hasGarden"].replace({True: 1}, inplace=True)


data.shape

(6731, 15)

In [253]:
ohe_cols=['type_of_property','bedroomCount','kitchen_type','fireplaceExists','hasTerrace','hasGarden']
for column in ohe_cols:

    print(data[column].value_counts())

APARTMENT    3531
HOUSE        3200
Name: type_of_property, dtype: int64
2.0    2772
3.0    1778
1.0    1419
4.0     565
5.0     158
6.0      35
7.0       4
Name: bedroomCount, dtype: int64
INSTALLED             2450
0                     2086
SEMI_EQUIPPED          983
HYPER_EQUIPPED         452
NOT_INSTALLED          428
USA_INSTALLED          151
USA_HYPER_EQUIPPED     117
USA_SEMI_EQUIPPED       60
USA_UNINSTALLED          4
Name: kitchen_type, dtype: int64
0    6616
1     115
Name: fireplaceExists, dtype: int64
1.0    3893
0.0    2838
Name: hasTerrace, dtype: int64
0    4966
1    1765
Name: hasGarden, dtype: int64


In [285]:
data2=data.sort_values(by=['price'])
data2.to_csv('datatowork.csv')

In [320]:
from sklearn.model_selection import ParameterGrid, train_test_split,cross_val_score,cross_validate

data2=pd.read_csv('datatowork.csv')
data2=pd.get_dummies(data, columns=['kitchen_type'])
data2=pd.get_dummies(data2, columns=['type_of_property'])


y=data2['price'].values
data2=data2.drop(['land_surface','terraceSurface','gardenSurface','facadeCount'],axis=1)
X=data2.drop(['price'],axis=1).values


AttributeError: 'numpy.ndarray' object has no attribute 'values'

array([[3., 0., 1., ..., 0., 0., 1.],
       [3., 0., 1., ..., 0., 0., 1.],
       [3., 0., 1., ..., 0., 0., 1.],
       ...,
       [3., 0., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [4., 0., 0., ..., 0., 0., 1.]])

In [287]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=0)
X



array([[3., 0., 1., ..., 0., 0., 1.],
       [3., 0., 1., ..., 0., 0., 1.],
       [3., 0., 1., ..., 0., 0., 1.],
       ...,
       [3., 0., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [4., 0., 0., ..., 0., 0., 1.]])

In [315]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
regr = make_pipeline(StandardScaler(), LinearRegression())
#regr = LinearRegression(normalize=True)
regr.fit(X_train, y_train)


#print(list(zip(regr['LinearRegression'].coef_, regr['LinearRegression'].feature_names_in_)))




[ 4.28118258e+03  1.65103351e+03  4.85699247e+03  1.71214779e+03
  1.89815552e+03  5.61688113e+03  1.26178276e+03  3.45829615e+03
  7.30839983e+17  3.92050145e+17  7.64317520e+17  3.89977507e+17
  5.58089423e+17  2.06365675e+17  2.32862618e+17  1.40984908e+17
  3.73782730e+16 -4.46142395e+17 -4.46142395e+17]


In [318]:
coef_table = pd.DataFrame(list(X.columns)).copy()


coef_table.insert(len(coef_table.columns),"Coefs",regr[1].coef_.transpose())



AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [291]:
import sklearn.metrics as metrics
print(regr.score(X_test, y_test))


0.12728360452959953


In [281]:
y_pred = regr.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df.to_csv('2.csv')



In [295]:
import numpy as np
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2 Score:', metrics.r2_score(y_test, y_pred))



Mean Absolute Error: 26049.94654788419
Mean Squared Error: 1024136041.3170007
Root Mean Squared Error: 32002.125574983307
R2 Score: 0.12693741695952898


In [293]:
print(data2['price'].mean())

193152.11736740454


In [296]:
import joblib
joblib.dump(regr, "clf.pkl")

['clf.pkl']

In [309]:
regr['linearregression'].__dict__

{'fit_intercept': True,
 'normalize': 'deprecated',
 'copy_X': True,
 'n_jobs': None,
 'positive': False,
 'n_features_in_': 19,
 'coef_': array([ 4.28118258e+03,  1.65103351e+03,  4.85699247e+03,  1.71214779e+03,
         1.89815552e+03,  5.61688113e+03,  1.26178276e+03,  3.45829615e+03,
         7.30839983e+17,  3.92050145e+17,  7.64317520e+17,  3.89977507e+17,
         5.58089423e+17,  2.06365675e+17,  2.32862618e+17,  1.40984908e+17,
         3.73782730e+16, -4.46142395e+17, -4.46142395e+17]),
 '_residues': 5552755417053.944,
 'rank_': 19,
 'singular_': array([1.39408985e+02, 9.12984134e+01, 8.43453233e+01, 7.95153110e+01,
        7.65763961e+01, 7.45674418e+01, 7.40334759e+01, 7.38668879e+01,
        7.35871628e+01, 7.33318276e+01, 7.32326330e+01, 7.00385893e+01,
        6.77893312e+01, 6.55630504e+01, 6.48077860e+01, 5.41781391e+01,
        4.04613268e+01, 5.32680120e-14, 4.08590699e-14]),
 'intercept_': 192737.04804622848}

In [321]:
data2

array([[3., 0., 1., ..., 0., 0., 1.],
       [3., 0., 1., ..., 0., 0., 1.],
       [3., 0., 1., ..., 0., 0., 1.],
       ...,
       [3., 0., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [4., 0., 0., ..., 0., 0., 1.]])